In [0]:
!pip install kafka-python
!pip install murmurhash

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
from kafka import KafkaProducer
import requests
import time
import datetime
import codecs

api_key = 'd65d7dda1cfde9f7a7f6f517d624d8a0c575f186647af620f9170e5bd5ef7092'
api_url = "https://min-api.cryptocompare.com/data/v2/news/?lang=EN"
producer = KafkaProducer(bootstrap_servers='localhost:9092')

## Aplicacion de filtro de Bloom en categoria de la noticas

```
Idea general
1.- Tomar las primeras 3 noticias y guardarlas en una lista de categorias inciales (estaticas)
2.- Leer las datos dinamicamente y extraer las categorías de las noticias
3.- Verificar si esa categoria ya esta en la categoria de la usuario.
4.- En caso de no estar la categoría guardar el título de la noticia en lista de recomendaciones
5.- Mostrar las recomendaciones
```

In [0]:
# Filtro de Bloom visto en clase

import numpy as np
import murmurhash
import warnings
warnings.filterwarnings('ignore')

class FiltroBloom:
    def __init__(self, n, m, k):
        self.n = n
        self.m = m
        self.k = k
        self.arreglo = np.zeros(n, dtype=np.bool)

    def construir(self, s):
        for i in range(self.k):
            # tener índices dentro del arreglo
            index = murmurhash.mrmr.hash(s, i) % self.n
            self.arreglo[index] = True

    def verifica(self, s):
        bits = np.zeros(self.k, dtype=np.bool)
        # print(bits)
        for i in range(self.k):
            index = murmurhash.mrmr.hash(s, i) % self.n
            bits[i] = self.arreglo[index]
            # print(bits)
        # si todos los elementos son "1" regresa True, si alguno de los elementos es "0" regresa "False"
        return np.all(bits)

In [0]:
#Creamos la lista que tendrá las categorías iniciales del usuario
categorias_usuario = []
#Creamos la lista que tendrá las recomendaciones
lista_recomendaciones = []

# Obtener categorias vistas por el usuario
response = requests.get(api_url)
json_ = response.json()
print("---Iniciando busqueda-----")  

#Obtenemos las primeras tres noticias del json y nos quedamos con las categorías de dichas noticias
for i in range(3):
    categorias = json_['Data'][i]['categories'].lower().split('|')
    categorias_usuario += categorias    

print("---Termino de encontrar categorias vistas por usuario-----")
# Eliminando categorias repetidas
categorias_usuario = list(set(categorias_usuario))
print(categorias_usuario)

# Instanciamos el filtro de bloom  y lo construimos
fb = FiltroBloom(1000, len(categorias_usuario), 3) #instancia de la clase (m,n,k)
#Recorremos la lista de URLs y las registramos
for u in categorias_usuario:
    fb.construir(u)

# Leer los datos dinamicamente
print("---Iniciando streaming-----") 

#Iniciamos nuestro loop para obtener nuevas noticias
while True:
    response = requests.get(api_url)
    json_ = response.json()    
    #De cada notica que llega obtenemos su categoría
    for i in range(len(json_['Data'])):        
        categorias_streaming = json_['Data'][i]['categories'].lower().split('|')                     
        print(f'Categoria recibida: {categorias_streaming}')
        # Aplicar el filtro de bloom, para verirficar si esta o no esta
        # En caso de no estar recomendamos la noticia al usuario  
        for categoria in categorias_streaming:                        
            if not fb.verifica(categoria):
                # Agregamos la noticia recomendada a nuestra lista_recomendaciones
                lista_recomendaciones.append(dict(categoria=categoria, titulo=json_['Data'][i]['title']))
                break
        time.sleep(2)        

In [0]:
#Imprimimos las noticias recomendadas al usuario
print("Noticias que recomendamos al usuario")
for recomendacion in lista_recomendaciones:
    print(f'Categoria: {recomendacion["categoria"]} \t\t\t Titulo: {recomendacion["titulo"][:100]}...') 

Noticias que recomendamos al usuario
Categoria: sponsored 			 Titulo: Is This Crypto Lending Company Any Good? Part 1...
Categoria: mining 			 Titulo: Block maps out path to growth through bitcoin, Tidal, Square and Cash App...
Categoria: blockchain 			 Titulo: Cointelegraph Research launches venture capital database...
Categoria: exchange 			 Titulo: Peckshield Sounds Alarm on Honeypot Posing as Terra Fork...
Categoria: exchange 			 Titulo: Meta May Launch Crypto Exchange, Trademark Filings Show...
Categoria: other 			 Titulo: Louisiana Considering Adopting Cryptocurrency...
Categoria: sponsored 			 Titulo: SneakMart Announces A Collaboration With StockX On It’s First Metakicks Drop...
Categoria: blockchain 			 Titulo: SneakMart Announces A Collaboration With StockX On It’s First Metakicks Drop...
Categoria: other 			 Titulo: The Evolution of Gaming and Its Integration into the Crypto Universe...


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e588c84d-c012-4c07-8593-b08d1bf1b087' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>